In [10]:
import pandas as pd
import numpy as np
import datetime

data_file = '../fxdata/EURUSD-2019-02.csv'

df = pd.read_csv(data_file, header=None)

df[1] = pd.to_datetime(df[1], format='%Y%m%d %H:%M:%S.%f')

df.head()

,0,1,2,3
0,EUR/USD,2019-02-01 00:00:00.334,1.14480,1.14485
1,EUR/USD,2019-02-01 00:00:00.414,1.14479,1.14485
2,EUR/USD,2019-02-01 00:00:00.535,1.14479,1.14481
3,EUR/USD,2019-02-01 00:00:00.546,1.14478,1.14482
4,EUR/USD,2019-02-01 00:00:00.605,1.14478,1.14480


In [18]:
base_date = datetime.datetime.strptime('2019-02-01 00:00:00', '%Y-%m-%d %H:%M:%S')
end_date = datetime.datetime.strptime('2019-03-01 00:00:00', '%Y-%m-%d %H:%M:%S')
date_increment = datetime.timedelta(hours=1)
current_date = base_date
hourly_df = []
while current_date < end_date:
    current_frame = { 'date': current_date }
    tickdata = df[(df[1] >= current_date) & (df[1] < (current_date + date_increment)) ][2]
    if tickdata.shape[0] == 0:
        current_date = current_date + date_increment
        continue
    current_frame['open'] = tickdata.iloc[0]
    current_frame['close'] = tickdata.iloc[-1]
    current_frame['high'] = tickdata.max()
    current_frame['low'] = tickdata.min()
    current_date = current_date + date_increment
    hourly_df.append(current_frame)
    
df = pd.DataFrame(hourly_df)

df.head()

,close,date,high,low,open
0,1.14482,2019-02-01 00:00:00,1.14499,1.14461,1.14480
1,1.14424,2019-02-01 01:00:00,1.14488,1.14376,1.14481
2,1.14426,2019-02-01 02:00:00,1.14431,1.14378,1.14424
3,1.14420,2019-02-01 03:00:00,1.14473,1.14404,1.14426
4,1.14419,2019-02-01 04:00:00,1.14435,1.14381,1.14419


In [20]:
# from here on it should look familiar
import pandas as pd
import numpy as np
import lightgbm as lg

# first let us create some fake price data
#n=10000
price_close = df['close']
price_open = df['open']
price_high = df['high']
price_low = df['low']

# the target is the next close price, lets construct some features - this is usually referred to as feature engineering
features = []
feature_length = 100
targets = price_close[feature_length:]
for i in range(feature_length, price_open.shape[0]):
    feature_object = dict()
    feature_object['MA'] = np.mean(price_open[(i-feature_length):i])
    feature_object['STD'] = np.std(price_open[(i-feature_length):i])
    feature_object['MIN'] = np.min(price_open[(i-feature_length):i])
    feature_object['MAX'] = np.max(price_open[(i-feature_length):i])
    features.append(feature_object)
df = pd.DataFrame(features)

In [21]:
import matplotlib.pyplot as plt 

plt.plot(price_close)

#plt.xlabel('')
#plt.ylabel('voltage (mV)')
#plt.title('About as simple as it gets, folks')
#plt.grid(True)
plt.savefig("test.png")
plt.show()

<Figure size 640x480 with 1 Axes>